In [66]:
import pandas as pd
import numpy as np
#import tensorflow as tf
from tqdm import tqdm
import feather

In [2]:
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import sys, os
import re
import json
from urllib.request import FancyURLopener
import cv2
import glob
import zipfile

In [7]:
KEY_FILE = '..//..//..//data//aws_keys//keys.txt'

f = open(KEY_FILE,'rb')
key_data = str(f.read())
keys = re.findall(pattern='\=([a-zA-Z0-9]*)',string=key_data)

AWS_KEY = keys[0]
AWS_SECRET = keys[1]

del keys

In [8]:
BUCKET = 'imaterialist-challenge'
PATH_TO_DATA = '..//..//..//data//imaterialist//'

In [10]:
json_files = ['fgvc4_iMat.label_map.json',
 'fgvc4_iMat.task_map.json',
 'fgvc4_iMat.train.data.json',
 'fgvc4_iMat.validation.data.json']

In [14]:
json_data = []

def de_dupe_list(seq): # Order preserving
    seen = set()
    return [x for x in seq if x not in seen and not seen.add(x)]

for file in tqdm(de_dupe_list(json_files)):
    try:
        json_data.append(json.loads(open(PATH_TO_DATA+file).read()))
    except:
        print('not json file')

100%|██████████| 4/4 [00:00<00:00, 10.91it/s]


In [15]:
[k for k in json_data[2]]

['annotations', 'info', 'license', 'images']

In [29]:
train_annotations = pd.DataFrame([ [ row['imageId'], row['labelId'], row['taskId'] ] for row in json_data[2]['annotations'] ])
train_annotations.columns = ['imageId','labelId','taskId']

In [30]:
train_annotations.head(10)

,imageId,labelId,taskId
0,1,6,5
1,2,7,6
2,3,8,7
3,4,10,9
4,5,14,12
5,5,15,6
6,6,16,7
7,7,23,6
8,8,27,13
9,8,28,17


In [31]:
[k for k in json_data[1]]

['taskInfo']

In [57]:
re.findall(pattern='[A-Za-z\s]*',string=)

['shoe', '', 'heel type', '']

In [60]:
task_df = pd.DataFrame([ [row['taskId'], row['taskName'], re.findall(pattern='[A-Za-z\s]*',string=row['taskName'])[0], re.findall(pattern='[A-Za-z\s]*',string=row['taskName'])[2] ] for row in json_data[1]['taskInfo']])
task_df.columns = ['taskId','taskName','taskName_type','taskName_subtype']

In [61]:
task_df.head(10)

,taskId,taskName,taskName_type,taskName_subtype
0,1,shoe:gender,shoe,gender
1,2,shoe:age,shoe,age
2,3,shoe:color,shoe,color
3,4,shoe:up height,shoe,up height
4,5,dress:decoration,dress,decoration
5,6,dress:color,dress,color
6,7,dress:material,dress,material
7,8,dress:silhouette,dress,silhouette
8,9,shoe:type,shoe,type
9,10,shoe:closure type,shoe,closure type


In [62]:
[k for k in json_data[0]]

['labelInfo']

In [63]:
json_data[0]['labelInfo']

[{'labelId': '1', 'labelName': 'men'},
 {'labelId': '2', 'labelName': 'adult'},
 {'labelId': '3', 'labelName': 'black'},
 {'labelId': '4', 'labelName': 'women'},
 {'labelId': '5', 'labelName': 'flat'},
 {'labelId': '6', 'labelName': 'printed'},
 {'labelId': '7', 'labelName': 'purple'},
 {'labelId': '8', 'labelName': 'velvet'},
 {'labelId': '9', 'labelName': 'mermaid'},
 {'labelId': '10', 'labelName': 'sneaker'},
 {'labelId': '11', 'labelName': 'slipon'},
 {'labelId': '12', 'labelName': 'light'},
 {'labelId': '13', 'labelName': 'sky blue'},
 {'labelId': '14', 'labelName': 'midi'},
 {'labelId': '15', 'labelName': 'navy'},
 {'labelId': '16', 'labelName': 'silk'},
 {'labelId': '17', 'labelName': 'beige'},
 {'labelId': '18', 'labelName': 'low'},
 {'labelId': '19', 'labelName': 'loose'},
 {'labelId': '20', 'labelName': 'blue'},
 {'labelId': '21', 'labelName': 'party'},
 {'labelId': '22', 'labelName': 'boot'},
 {'labelId': '23', 'labelName': 'red'},
 {'labelId': '24', 'labelName': 'lace'},
 {

In [64]:
label_df = pd.DataFrame([ [ row['labelId'], row['labelName'] ] for row in json_data[0]['labelInfo'] ])
label_df.columns = ['labelId','labelName']

In [65]:
label_df.head(10)

,labelId,labelName
0,1,men
1,2,adult
2,3,black
3,4,women
4,5,flat
5,6,printed
6,7,purple
7,8,velvet
8,9,mermaid
9,10,sneaker


# Now write dfs to file and s3.

In [70]:
feather.write_dataframe(label_df,PATH_TO_DATA+'labeldf.feather')
feather.write_dataframe(task_df,PATH_TO_DATA+'taskdf.feather')
feather.write_dataframe(train_annotations,PATH_TO_DATA+'train_annotations.feather')

In [69]:
conn = S3Connection(AWS_KEY, AWS_SECRET)
bucket = conn.get_bucket(BUCKET)

In [71]:
files = [PATH_TO_DATA+'labeldf.feather',PATH_TO_DATA+'taskdf.feather',PATH_TO_DATA+'train_annotations.feather']

In [75]:
to_s3_file = lambda x: re.findall(pattern='[A-Za-z_]*.feather',string=x)[0]

In [79]:
# yolo: transfer remaining files in.
for sample_file in tqdm(files):
    s3_file = to_s3_file(sample_file)
    k = Key(bucket)
    k.key = s3_file
    k.set_contents_from_filename(filename=sample_file)

100%|██████████| 3/3 [00:03<00:00,  1.19s/it]
